# Household Debt Inequalities

A relatively simple two sheet extraction from data first published by the UK Office for National Statistics.

## Tutorial Structure

With these example tutorials I'm going to comment heavily and cover nuances in a follow up section (with liberal targetted previews as needed) as it's the easiest way to grapple with new ideas. It may also be worth opening up these notebooks yourself (they're in `./jupyterbook` in the [tidychef](https://github.com/mikeAdamss/tidychef) github repo) so you can run, alter and generally have a play about with this yourself as part of your learning.

We'll cover:

- source data
- requirements, what we're aiming to do here
- show the full script (all logic commented)
- output the selection preview
- nuances (where applicable)
- view the output

This sequencing is necessary as the output for some of the example is **really** long so that necessitates it coming last. If you're viewing this via a jupyter book (i.e on the site) you can navigate between the above sections via your right hand menu.

_Note - these tutorial scripts might seem verbose due all the comments but that's ok (this is a tutorial after all). If you take them out you end up with a fairly succinct and human readable encapsulation of what would otherwise (with existing tools) be a rather convoluted and fragile set of instructions to express._

In virtually all cases I'll make heavy use of `preview` and `bounded` to only look at relevant parts of what can be quite large datasets. Downloads links are provided for the source data.

## Source

For this example we're extracting the tables 11 and 12 from an xls dataset dealing with household debt inequalities.

The example highlights using iteration to join multiple tables into a coherent whole.

In [ ]:
from typing import List
from tidychef import acquire, preview
from tidychef.selection import XlsSelectable

tables: List[XlsSelectable] = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/householddebtdataset.xls", tables="Table 11|Table 12")
for table in tables:
    preview(table)

From an xls source which can be [downloaded here](https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/householddebtdataset.xls).

## Requirements

- We're going to extract "Period" from the obvious dates in column A.
- We're just going to call the principle field indicated by column A "Category".
- We're going to take "Great Britain" as a constant for a column named "Area".
- We're going to take the headers on row 4 as "Financial Liability"
- As an additional exercise we're going to use a horizontal condition to create a "Unit Of Measure" column to be one of "Pounds Sterling", "Percent", "Ratio" or "Number" depending on the category.
- We're going to prefix "Category" as extracted from table 12 with "Education: " to make the data a little easier to understand.
- We're going to join both tables into a single tidy data output.
- We're going to de-duplicate with a printout of what we've removed - it should be the contents of row 14 as its duplicated on both tables.
- We'll strip trailing ".0"s from the observations (which we'll call "Value" this time).

In [ ]:
from typing import Dict, List
from tidychef import acquire, preview
from tidychef.direction import down, right, left
from tidychef.output import Column, TidyData
from tidychef.selection import XlsSelectable

# We'll use the vategory information to determine the unit of measure.
# For more on how this works see the documentation for horizontal condition and the tourism example.
def derive_unit_of_measure(line) -> str:
    """
    Function to define unit of measure based on Financial Liability
    """
    cat = line["Category"]
    if "(%)" in cat:
        return "Percent"
    elif "(£)" in cat:
        return "Pounds Sterling"
    elif "Frequency" in cat:
        return "Number"
    elif "Ratio" in cat:
        return "Ratio"
    else:
        raise Exception(f"Cannot identify unit of measure from: {cat}")

# We'll use two tables from this data source.
tables: List[XlsSelectable] = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/householddebtdataset.xls", tables="Table 11|Table 12")

all_tidy_data = []
for table in tables:
    
    # We just want the cell with Great Britain in it.
    # You could hard code this into the tidydata but this way we'll be warned if this area cell is not there 
    area = table.excel_ref("A").cell_containing_string("Great Britain").label_as("Area")
    
    # We'll need a regex to get the periods out of column A. This regex is looking for any string ending in 4 integers, i.e a year
    period = table.excel_ref("A3").fill(down).re(".*[0-9]{4}").assert_len(2).label_as("Period")
    
    # The categories are to one level down and to the right of the area ell
    category = area.shift(down).fill(right).label_as("Category")
    
    # Observations are any numeric values below a category
    observations = category.fill(down).is_not_blank().is_numeric().label_as("Observation")
    
    # See nuances
    financial_liability = (observations.shift(left) - observations).label_as("Financial Liability")
    
    # The table in question dictates the type of liability category
    liability_category = "Economic Activity" if table.name == "Table 11" else "Education Level"
    
    preview(observations, area, period, category, financial_liability)

    tidy_data = TidyData(
        observations,
        Column(period.attach_closest(down)),
        Column(category.attach_directly(down), apply=lambda x: "Eduction: "+x if table.name == "Table 12" else x), # see nuances
        Column(financial_liability.attach_directly(right)),
        Column.constant("Area", area.lone_value()),
        Column.constant("Liability Category", liability_category),
        Column.horizontal_condition("Unit Of Measure", derive_unit_of_measure), # Derive unit of measure
        obs_apply=lambda x: x.replace(".0", "") # We'll clean off the unwanted '.0's from the observations. 
    )

    all_tidy_data.append(tidy_data)

final_tidy_data = TidyData.from_tidy_list(all_tidy_data)

## Nuances

### Shift and remove self

So there's a slightly counter intuitive pattern I'm using here that's worth exploring, namely:

```python
financial_liability = (observations.shift(left) - observations).label_as("Financial Liability")
```

To understand this, we'll look at a few rows only of both `observations` and `observations.shift(left)`


In [ ]:
preview(observations.shift(left).label_as("Observations shifted left"), bounded="A5:H8")
preview(observations.label_as("Observations"), bounded="A5:H8")

Now lets look at what you get if you remove the first from the second

In [ ]:
result = (observations.shift(left) - observations).label_as("Result of subrtraction")
preview(result, bounded="A5:H8")

In simplest terms a subtraction of selections equates to

**Remove any cells in the 2nd selection from the cells in the 1st selection.**

Which is exactly what we're doing above.

# Outputs

The tidy data can be [downloaded here](./household-debt.csv) and a full inline preview of the tidydata generated is shown below for those people who'd prefer to scroll.

In [ ]:
print(final_tidy_data)